# TenrFlow框架下的CNN构建说明  
   by 徐诗乐我一定写的比CDSN上的好！算了随便写写

## 说明
1. 本文只讨论2d情况

## 总览

1. 卷积层(convolution) tf.nn.conv2d()  
2. (可选) 批归一化(batch normalization) tf.nn.batch_normalization()  
3. (可选) 激活函数(activation)(一般使用ReLU) tf.nn.relu()  
4. （最大）池化(1-max-pooling) tf.nn.max_pool()  
5. 全连接层(full connect) ( 即对上面的输出进行：ReLU(wx+b) ) tf.nn.xw_plus_b() tf.nn.relu()  
6. (可选) dropout tf.nn.dropout()  
  
   由最后输出的logits，进入softmax函数，计算交叉熵，并计算l2_loss，构造损失函数
7. 损失函数(loss fuction)  
   losses = cross_entropy + l2_loss
  * softmax function和cross entropy tf.nn.softmax_cross_entropy_with_logits()  
  * (可选) l2 loss (待完善)  
  

## 细节说明
1. 卷积层(convolution)  
   [tf.nn.conv2d(x_input, filter, strides, padding)](https://tensorflow.google.cn/api_docs/python/tf/nn/conv2d)  
   
   * x_input 输入  
     * shape=\[batch_size, input_height, input_width, input_channels\]  
     * channels可以理解为输入的通道数，类似RGB，卷积时通常会在这一层也进行卷积，并且窗口大小与channels数相同（详见[Stanford深度学习视频](https://www.bilibili.com/video/av17204303/?p=12)）  
  
   * filter 卷积核 tf.get_variable()  
     * shape=\[filter_height, filter_width, input_channles, output_channels\]  
     * input_channels 为函数输入的input_x的channels  
     * output_channels(=filter_number) 卷积后输出的channels数=这层卷积的卷积核数量  
   
   * strides 卷积核在x_input上每一维的移动步长，shape对应x_input的每一维度， 是一个 length=4 的列表，如下所示
     * \[1, stride_of_height, stride_of_width, 1\]
     * 一般在batch和channel维度不做stride，因此均为1  
   
   * padding 卷积是否边缘补0  
     * padding='VALID' 不考虑边缘，直接卷积，输出shape如下：(参考[官方说明文档Convolutian部分](https://tensorflow.google.cn/api_guides/python/nn#Convolution))  
       out_height = ceil(float(in_height - filter_height + 1) / float(strides\[1\]))  
       out_width  = ceil(float(in_width - filter_width + 1) / float(strides\[2\]))  
     * padding='SAME' 边缘补0，一般来说，输出与输入的形状相同，输出shape如下：  
       out_height = ceil(float(in_height) / float(strides\[1\]))  
       out_width  = ceil(float(in_width) / float(strides\[2\]))  
  
2. 批归一化(batch normalization)  
   [tf.nn.batch_normalization(x, mean, variance, offset, scale,variance_epsilon, name=None)](https://tensorflow.google.cn/api_docs/python/tf/nn/batch_normalization)  
   [tf.nn.moment(x, axes)](https://tensorflow.google.cn/api_docs/python/tf/nn/moments)  
   
   tf.nn.batch_normalization(x, mean, variance, offset, scale,variance_epsilon, name=None)  
   函数输出的公式：$\frac{\gamma(x-\mu)}{\sigma}+\beta$  
   * x 输入
   * mean, variance 为平均值与方差，对应公式中的$\mu,\sigma^2$，由tf.nn.moment()生成
   * offset, scale 对应公式中的$\beta,\gamma$，可以为None
   * variance_epsilon 即希腊字母$\epsilon$，是一个加在方差$\sigma^2$上的较小的数，避免方差为0的情况下出问题
   
   tf.nn.moments(x, axes)
   * x 输入
   * axes Array of ints. Axes along which to compute mean and variance.
   * 输出 = mean, variance
   
3. 激活函数(activation)(一般使用ReLU)  
   [tf.nn.relu(features, name=None)](https://tensorflow.google.cn/api_docs/python/tf/nn/relu)
   
   * features (A Tensor)
   
4. （最大）池化(1-max-pooling)   
    [tf.nn.max_pool(value, ksize, strides, padding, data_format='NHWC', name=None)](https://tensorflow.google.cn/api_docs/python/tf/nn/max_pool)  
    
    * value (A Tensor)
    * ksize 窗口大小
      \[1, ksize_height, ksize_width, 1\]
    * strides 池化步长，一般与ksize相同
    * padding
     * ='VALID'
     * ='SAME'
    
5. 全连接层(full connect)   
   [tf.nn.xw_plus_b(x, weights, biases, name=None)](https://tensorflow.google.cn/api_docs/python/tf/nn/xw_plus_b)  
   [tf.nn.relu(features, name=None)](https://tensorflow.google.cn/api_docs/python/tf/nn/relu)   
   
   公式为：$ReLU(wx+b)$  
   也可以使用ReLU以外的激活函数，如sigmoid, tanh  
   
6. dropout  
   [tf.nn.dropout(x, keep_prob)](https://tensorflow.google.cn/api_docs/python/tf/nn/dropout)  
   
   * x (A Tensor)
   * keep_prob 未置0的数据的比例
   
7. 损失函数(loss fuction)  

  * softmax function和cross entropy  
   [tf.nn.softmax_cross_entropy_with_logits(logits, labels)](https://tensorflow.google.cn/api_docs/python/tf/nn/softmax_cross_entropy_with_logits)  
   以上函数基于logits计算交叉熵(cross entropy)（即该函数包含了：1.softmax层；2.计算cross entropy，即损失），可能会加上l2 loss  
   
    * logits 输入
    * labels 标签  
  
  * l2 loss